In [1]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *

In [2]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


In [3]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


In [4]:
# printing the shapes of the arrays
print("the shape of the training y is ",y_tr.shape," and the training tx is ", tx_tr.shape)
print("the shape of the test y is ",y_te.shape," and the test tx is ",tx_te.shape)

the shape of the training y is  (250000,)  and the training tx is  (250000, 30)
the shape of the test y is  (568238,)  and the test tx is  (568238, 30)


In [9]:
# testing the mean square gd
initial_w = np.zeros(tx_tr.shape[1])
max_iters = 500
gamma = 0.00008

w, loss = mean_squared_error_gd(y_tr, tx_tr, initial_w, max_iters, gamma)

print("end of the mean_squared_error_gd with w=",w," and loss=", loss)

Current iteration=0, the loss=12796.178310949084, the grad=-83.42721036786665
Current iteration=1, the loss=2680491373.0276556, the grad=37489.38639399656
Current iteration=2, the loss=561706185275329.8, the grad=-17128797.794126377
Current iteration=3, the loss=1.177076162364914e+20, the grad=7840049095.731387
Current iteration=4, the loss=2.4666068135676807e+25, the grad=-3588907012061.712
Current iteration=5, the loss=5.168866185241528e+30, the grad=1642893992715460.5
Current iteration=6, the loss=1.0831551057908984e+36, the grad=-7.520681452936573e+17
Current iteration=7, the loss=2.269791751527176e+41, the grad=3.4427451308580327e+20
Current iteration=8, the loss=4.756432913215095e+46, the grad=-1.5759867201147976e+23
Current iteration=9, the loss=9.967281819001249e+51, the grad=7.214400278342999e+25
Current iteration=10, the loss=2.0886809227009536e+57, the grad=-3.3025387024973844e+28
Current iteration=11, the loss=4.376908445127169e+62, the grad=1.5118043719125051e+31
Current i

/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Current iteration=57, the loss=inf, the grad=3.7092741048341244e+153


/home/ml-project-1-ahl/helper_functions.py:25: RuntimeWarning: overflow encountered in square
  return (1 / 2) * np.mean(e**2)
/home/ml-project-1-ahl/implementations.py:36: RuntimeWarning: invalid value encountered in double_scalars
  if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:


Current iteration=58, the loss=inf, the grad=-1.6979957885341552e+156
Current iteration=59, the loss=inf, the grad=7.772921645564562e+158
Current iteration=60, the loss=inf, the grad=-3.5582132368092603e+161
Current iteration=61, the loss=inf, the grad=1.6288445987139566e+164
Current iteration=62, the loss=inf, the grad=-7.456367986362624e+166
Current iteration=63, the loss=inf, the grad=3.41330435033213e+169
Current iteration=64, the loss=inf, the grad=-1.5625096037782444e+172
Current iteration=65, the loss=inf, the grad=7.152706032972663e+174
Current iteration=66, the loss=inf, the grad=-3.274296904826221e+177
Current iteration=67, the loss=inf, the grad=1.4988761136739909e+180
Current iteration=68, the loss=inf, the grad=-6.861410768311749e+182
Current iteration=69, the loss=inf, the grad=3.140950563025931e+185
Current iteration=70, the loss=inf, the grad=-1.4378341091216056e+188
Current iteration=71, the loss=inf, the grad=6.581978556713927e+190
Current iteration=72, the loss=inf, 

KeyboardInterrupt: 

In [6]:
# test least square
w, loss = least_squares(y_tr, tx_tr)
print("end of the least_squares with w=",w," and loss=", loss)

end of the least_squares with w= [ 8.03494312e-05 -7.20202273e-03 -6.05417273e-03 -5.47559065e-04
 -1.93874700e-02  4.73451621e-04 -2.60379054e-02  3.25106300e-01
 -3.80780282e-05 -2.72724919e+00 -2.21220140e-01  9.50794091e-02
  6.40351613e-02  2.73550887e+00 -3.31801241e-04 -9.54325120e-04
  2.74026561e+00 -5.34164891e-04  9.73498581e-04  3.69225052e-03
  3.54487449e-04 -5.43344598e-04 -3.30448035e-01 -1.40800498e-03
  8.31432888e-04  1.02117272e-03 -1.68047416e-03 -5.83664818e-03
 -1.11087997e-02  2.72770912e+00]  and loss= 0.3396868094770935


In [7]:
# test mean squared error sgd

initial_w = np.zeros(tx_tr.shape[1])
max_iters = 100
gamma = 0.00001

#w, loss = mean_squared_error_sgd(y_tr, tx_tr, initial_w, max_iters, gamma)

print("end of the mean_squared_error_sgd with w=",w," and loss=", loss)

end of the mean_squared_error_sgd with w= [ 8.03494312e-05 -7.20202273e-03 -6.05417273e-03 -5.47559065e-04
 -1.93874700e-02  4.73451621e-04 -2.60379054e-02  3.25106300e-01
 -3.80780282e-05 -2.72724919e+00 -2.21220140e-01  9.50794091e-02
  6.40351613e-02  2.73550887e+00 -3.31801241e-04 -9.54325120e-04
  2.74026561e+00 -5.34164891e-04  9.73498581e-04  3.69225052e-03
  3.54487449e-04 -5.43344598e-04 -3.30448035e-01 -1.40800498e-03
  8.31432888e-04  1.02117272e-03 -1.68047416e-03 -5.83664818e-03
 -1.11087997e-02  2.72770912e+00]  and loss= 0.3396868094770935


In [8]:
# test ridge regression 

#initial_w = np.zeros(tx_tr.shape[1])
#max_iters = 100
#lambda_ = 0.0005

#w, loss = ridge_regression(y_tr, tx_tr, lambda_)
lambda_ = ridge_regression_best_lambda(y_tr, tx_tr)
print("best lambda is ",lambda_)
w, loss = ridge_regression(y_tr, tx_tr, lambda_)

#y_pred = predict(w, tx_te)
#y_pred[y_pred==0] = -1

#OUTPUT_PATH = 'sample-submission'
#create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

print("end of the ridge_regression with w=",w," and loss=", loss)

Current lambda=0.0001, the loss=0.33968798676609097
Current lambda=0.00013738237958832623, the loss=0.33968803954866844
Current lambda=0.00018873918221350977, the loss=0.33968813859041375
Current lambda=0.0002592943797404667, the loss=0.3396883245147402
Current lambda=0.0003562247890262444, the loss=0.3396886731920951
Current lambda=0.0004893900918477494, the loss=0.3396893258059021
Current lambda=0.0006723357536499335, the loss=0.33969054373021085
Current lambda=0.0009236708571873865, the loss=0.33969280743924646
Current lambda=0.0012689610031679222, the loss=0.3396969916896993
Current lambda=0.0017433288221999873, the loss=0.33970466833847507
Current lambda=0.002395026619987486, the loss=0.3397186122395501
Current lambda=0.0032903445623126675, the loss=0.33974360637308687
Current lambda=0.004520353656360241, the loss=0.33978763536530615
Current lambda=0.006210169418915616, the loss=0.3398634706893294
Current lambda=0.008531678524172805, the loss=0.33999040783666146
Current lambda=0.0

In [ ]:
# test logistic regression 
